# Causal Injection into a Neural Network

## Reconstruction experiment

In [166]:
#Run a main_biggerDAG_extended.py
rerun_experiment = True
if rerun_experiment:
    !which python
    # !python 'test.py' #main_biggerDAG_extended.py
    %run -i 'test.py'

/vol/bitbucket/fr920/envs/bayesenv/bin/python
Hello capo
Version: None
csv: None

Num GPUs Available:  0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


### Recostruction Experiment 20% known

In [167]:
import re
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import load_pickle, random_stability

verbose = False
verbose2 = False
nodes = '([\w]+)'
seeds = '([\w]+)'
dsets = '([\w.]+)'
outfs = str(1)
infs = str(1)
thetas = '([-+]?([0-9]+)(\.[0-9]+)?)'

# ## Versions
# # 'v3' reconstrunction toy experiment - 84%
# # 'p1' = partial toy experiment - 96%
# # 'r2' = reconstruction toy experiment, 20n 4b - 0%
# # 'r3' = reconstruction toy experiment, 10n 4b - 0%
# # 'r4' = reconstruction toy experiment, 10n 1b - 0%
# # 'r5' = reconstruction toy experiment, 15n 2b - ?
# # 'r6' = reconstruction toy experiment, 10n 1b - 32hidden
# # 'r7' = reconstruction toy experiment, 20n 4b - 0%
# # 'r_ex3' = reconstruction experiment, 10n 20n 50n 1b
# # 'r_ex_2b' = reconstruction experiment, 10n 20n 50n 2b
# # 'r_ex_5b' = reconstruction experiment, 10n 20n 50n 5b

version = '(r_ex3|r_ex_2b|r_ex_5b)'

known_p = 0.2

folder = './results/'
# CASTLE/results/Nested1FoldCASTLE.Reg.Synth.10.625.r_ex2.pkl

count = 0
res = []

for filename in os.listdir(folder):
    match = re.search(f'Nested1FoldCASTLE.Reg.Synth.{nodes}.{dsets}.{version}.pkl$', filename)
    if match != None:
        count += 1
        
        report = load_pickle(os.path.join(folder, filename), verbose=False)
        for key, value in report.items():
            # print(key, value)
            t = [[match.group(3)],[value[i] for i in ['theta', 'n_nodes', 'N_edges', 'seed', 'data_size', 'MSE', 'MAE','right','matching']]]
            res.append([i for sublist in t for i in sublist])
            # print(res)
print("Count=",count)

df = pd.DataFrame(res)
df.columns = ['V','Type', 'N_nodes', 'N_edges', 'seed', 'Size', 'MSE', 'MAE','right','matching']
df['alpha'] = ((df['Size'].astype(int)*0.8)/df['N_nodes'].astype(int)).astype(int)
df['branch'] = np.round(((df['N_edges'].astype(int))/df['N_nodes'].astype(int)).astype(float),0).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 5)])
df['branch_bin'] = pd.cut(df['branch'], bins)

df['alpha2'] = ((df['Size'].astype(int)*0.8)/df['branch'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 1000), (1000, 2000), (2000, 5000), (5000,np.Inf)])
df['alpha3'] = pd.cut(df['alpha2'], bins)
df['rebase_den'] = (df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))
df['rebased'] = (df['matching'].astype(int)/(df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))).clip(upper= 1 )
# display(df)

df = df[df['alpha']!=1000]

def plot_ly_by(x, df, x_desc):
    df = df.sort_values(by=[x],axis=0)
    # df=df[['Type', 'N_nodes','alpha', 'MSE', 'MAE','right']]
    mses = df.groupby(['Type',x], as_index=False).agg([ 'count','mean','std']).round(2).reset_index()
    mses.columns = list(map(''.join, mses.columns.values))
    mses['text'] = mses[['MSEmean','MSEstd']].apply(lambda x : '{} ({})'.format(x[0],x[1]), axis=1)#mses['MSEmean'].astype(str) + mses['MSEstd'].astype(str) 
    mses[x] = mses[x].astype(str)
    display(mses)

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    msesC = mses[mses['Type']==-1]
    fig.add_trace(
        go.Bar(x=msesC[x], y=msesC['MSEmean'], name="CASTLEMSE",
        marker_color='#595959',#'#FF4136', 
        marker_line_color='#262626',#rgb(8,48,107)',
                    marker_line_width=2, opacity=0.6,
        # error_y=dict(
        #         type='data', # value of error bar given in data coordinates
        #         array=msesC['MSEstd'],
        #         visible=True,
        #         color='Black',
        #         thickness=1.5,
        #         width=3),
        showlegend=False,
            text=msesC['text'],
            textposition='auto',
                    ),
        secondary_y=True,
    )
    msesI = mses[mses['Type']==0.05]
    fig.add_trace(
        go.Bar(x=msesI[x], y=msesI['MSEmean'], name="InjectedMSE",
        marker_color='#005383',#'#0085CA',#'#3D9970', 
        marker_line_color='#001F3A',#'#0085CA',#'White',
                    marker_line_width=2, opacity=0.6,
        # error_y=dict(
        #         type='data', # value of error bar given in data coordinates
        #         array=msesI['MSEstd'],
        #         visible=True,
        #         color='Black',
        #         thickness=1.5,
        #         width=3),
        showlegend=False,
            text=msesI['text'],
            textposition='inside',
                    ),
        secondary_y=True,
    )

    fig.add_trace(go.Box(
        y=df[df['Type']==-1]['right'],
        x=df[df['Type']==-1][x].astype(str),
        boxmean='sd', # represent mean
        boxpoints=False,#'outliers',
        name='CASTLE+',
        marker_color='#595959'#'#FF4136'
    ),
        secondary_y=False,)

    d = msesC['rebasedmean']#[i*1.2 for i in msesC['rightmean']]

    fig.add_trace(
        go.Scatter(x=msesC[x], y=d, name="CASTLE+20%" , mode='markers', marker_symbol='line-ew',
                marker=dict(
                color='Black',
                size=100,
                line=dict(
                    color='Black',
                    width=2)
                    ),
                showlegend=False),
        secondary_y=False, 
    )


    fig.add_trace(go.Box(
        y=df[df['Type']==0.05]['right'],
        x=df[df['Type']==0.05][x].astype(str),
        boxmean='sd', # represent mean
        boxpoints=False,#'outliers',
        name='Injected',
        marker_color='#005383'#'#0085CA'#'#3D9970'
    ),
        secondary_y=False,)
    fig.update_layout(
        # yaxis_title='normalized moisture',
        boxmode='group' # group together boxes of the different traces for each value of x
    )
    # Add figure title
    fig.update_layout(
        margin=dict(l=50, r=0, t=20, b=20),
        title={
            'text': "",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        legend={
            'y':-0.17,
            'x':0.85,
            'orientation':"h",
            'xanchor': 'center',
            'yanchor': 'bottom'},
        template='plotly_white',
            font=dict(
                # family="Courier New, monospace",
                size=18,
                # color="Black"
            )
    )
    # Set x-axis title
    fig.update_xaxes(showgrid=True,title={'text':x_desc,'font':{'size':18}})
    # Set y-axes titles
    fig.update_yaxes(showgrid=True,nticks=10,zeroline=True, title={'text':"% of right edges",'font':{'size':18}}, 
    range=[-0.3, 1],
    tickvals=[0,0.2,0.4,0.6,0.8,1],
    tickformat=".0%",
    secondary_y=False
    )
    fig.update_yaxes(showgrid=False,nticks=10,zeroline=False, title={'text':"Mean Squared Error",'font':{'size':18}},
    range=[0, 5],
    secondary_y=True)
    fig.show()
    
plot_ly_by('alpha', df, x_desc="s = N/|V| (Dataset Size / Number of Nodes)")

plot_ly_by('N_nodes', df, x_desc="|V| = Number of Nodes in G")

plot_ly_by('N_nodes', df[df['alpha'] == 100], x_desc="|V| = Number of Nodes in G")

plot_ly_by('branch', df, x_desc="b = |E|/|V| (Number of Edges / Number of Nodes)")
plot_ly_by('branch_bin', df, x_desc="b = |E|/|V| (Number of Edges / Number of Nodes)")

plot_ly_by('alpha3', df, x_desc="Dataset Size / b (b=N Edges/N Nodes)")


# plot_ly_by('Size', df[df['N_nodes'] == 10], x_desc="Dataset Size")

Count= 48


,Type,alpha,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha2count,alpha2mean,alpha2std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,50,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,772.61,701.91,90,51.03,51.69,90,0.17,0.22,0.62 (0.43)
1,-1.00,100,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,1545.33,1403.73,90,51.03,51.69,90,0.34,0.23,0.53 (0.36)
2,-1.00,200,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,3090.67,2807.47,90,51.03,51.69,90,0.54,0.22,0.5 (0.33)
3,-1.00,300,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,4636.11,4211.10,90,51.03,51.69,90,0.54,0.23,0.5 (0.33)
4,-1.00,500,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,7726.78,7018.57,90,51.03,51.69,90,0.55,0.23,0.49 (0.33)
5,0.05,50,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,772.61,701.91,90,51.03,51.69,90,0.27,0.26,0.6 (0.42)
6,0.05,100,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,1545.33,1403.73,90,51.03,51.69,90,0.51,0.23,0.51 (0.33)
7,0.05,200,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,3090.67,2807.47,90,51.03,51.69,90,0.57,0.23,0.5 (0.33)
8,0.05,300,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,4636.11,4211.10,90,51.03,51.69,90,0.56,0.24,0.5 (0.33)
9,0.05,500,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,7726.78,7018.57,90,51.03,51.69,90,0.58,0.22,0.5 (0.33)


,Type,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,Sizemean,...,alpha2count,alpha2mean,alpha2std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,10,150,19.47,9.01,150,2811,2477.95,150,2875,...,150,1405.40,1263.66,150,16.0,7.33,150,0.47,0.30,0.46 (0.36)
1,-1.00,20,150,45.60,25.61,150,2811,2477.95,150,5750,...,150,2683.33,2565.36,150,36.9,20.57,150,0.35,0.28,0.55 (0.37)
2,-1.00,50,150,124.77,75.42,150,2811,2477.95,150,14375,...,150,6574.17,6472.14,150,100.2,60.27,150,0.46,0.22,0.57 (0.34)
3,0.05,10,150,19.47,9.01,150,2811,2477.95,150,2875,...,150,1405.40,1263.66,150,16.0,7.33,150,0.56,0.28,0.46 (0.36)
4,0.05,20,150,45.60,25.61,150,2811,2477.95,150,5750,...,150,2683.33,2565.36,150,36.9,20.57,150,0.43,0.27,0.54 (0.36)
5,0.05,50,150,124.77,75.42,150,2811,2477.95,150,14375,...,150,6574.17,6472.14,150,100.2,60.27,150,0.50,0.21,0.56 (0.32)


,Type,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,Sizemean,...,alpha2count,alpha2mean,alpha2std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,10,30,19.47,9.13,30,2811,2511.89,30,1250,...,30,611.00,288.23,30,16.0,7.43,30,0.35,0.24,0.47 (0.37)
1,-1.00,20,30,45.60,25.96,30,2811,2511.89,30,2500,...,30,1166.67,634.27,30,36.9,20.86,30,0.19,0.12,0.56 (0.38)
2,-1.00,50,30,124.77,76.45,30,2811,2511.89,30,6250,...,30,2858.33,1651.39,30,100.2,61.09,30,0.48,0.21,0.56 (0.33)
3,0.05,10,30,19.47,9.13,30,2811,2511.89,30,1250,...,30,611.00,288.23,30,16.0,7.43,30,0.55,0.24,0.45 (0.35)
4,0.05,20,30,45.60,25.96,30,2811,2511.89,30,2500,...,30,1166.67,634.27,30,36.9,20.86,30,0.46,0.22,0.51 (0.32)
5,0.05,50,30,124.77,76.45,30,2811,2511.89,30,6250,...,30,2858.33,1651.39,30,100.2,61.09,30,0.51,0.22,0.56 (0.33)


,Type,branch,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha2count,alpha2mean,alpha2std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,1,150,26.67,17.05,150,26.20,16.89,150,2811.00,...,150,6133.33,6415.34,150,21.27,13.55,150,0.52,0.31,0.6 (0.39)
1,-1.00,2,150,26.67,17.05,150,50.77,34.27,150,2811.00,...,150,3066.67,3207.67,150,41.10,27.45,150,0.42,0.25,0.56 (0.33)
2,-1.00,3,50,10.00,0.00,50,31.20,2.01,50,2811.00,...,50,766.20,538.75,50,25.50,1.64,50,0.51,0.25,0.27 (0.21)
3,-1.00,4,65,26.92,12.74,65,113.31,61.20,65,2708.46,...,65,1548.08,1404.98,65,91.08,48.80,65,0.24,0.15,0.61 (0.37)
4,-1.00,5,35,50.00,0.00,35,228.71,1.86,35,3001.43,...,35,2300.00,1623.36,35,183.29,1.51,35,0.26,0.12,0.31 (0.23)
5,0.05,1,150,26.67,17.05,150,26.20,16.89,150,2811.00,...,150,6133.33,6415.34,150,21.27,13.55,150,0.61,0.28,0.59 (0.38)
6,0.05,2,150,26.67,17.05,150,50.77,34.27,150,2811.00,...,150,3066.67,3207.67,150,41.10,27.45,150,0.51,0.23,0.55 (0.33)
7,0.05,3,50,10.00,0.00,50,31.20,2.01,50,2811.00,...,50,766.20,538.75,50,25.50,1.64,50,0.53,0.25,0.27 (0.21)
8,0.05,4,65,26.92,12.74,65,113.31,61.20,65,2708.46,...,65,1548.08,1404.98,65,91.08,48.80,65,0.29,0.14,0.59 (0.35)
9,0.05,5,35,50.00,0.00,35,228.71,1.86,35,3001.43,...,35,2300.00,1623.36,35,183.29,1.51,35,0.30,0.09,0.31 (0.22)


,Type,branch_bin,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha2count,alpha2mean,alpha2std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,"(0, 1]",150,26.67,17.05,150,26.20,16.89,150,2811,...,150,6133.33,6415.34,150,21.27,13.55,150,0.52,0.31,0.6 (0.39)
1,-1.00,"(1, 2]",150,26.67,17.05,150,50.77,34.27,150,2811,...,150,3066.67,3207.67,150,41.10,27.45,150,0.42,0.25,0.56 (0.33)
2,-1.00,"(2, 5]",150,26.67,17.05,150,112.87,83.68,150,2811,...,150,1462.90,1369.43,150,90.73,66.82,150,0.34,0.22,0.43 (0.33)
3,0.05,"(0, 1]",150,26.67,17.05,150,26.20,16.89,150,2811,...,150,6133.33,6415.34,150,21.27,13.55,150,0.61,0.28,0.59 (0.38)
4,0.05,"(1, 2]",150,26.67,17.05,150,50.77,34.27,150,2811,...,150,3066.67,3207.67,150,41.10,27.45,150,0.51,0.23,0.55 (0.33)
5,0.05,"(2, 5]",150,26.67,17.05,150,112.87,83.68,150,2811,...,150,1462.90,1369.43,150,90.73,66.82,150,0.37,0.21,0.42 (0.32)


,Type,alpha3,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha2count,alpha2mean,alpha2std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,"(0.0, 500.0]",87,16.67,10.96,87,53.39,57.79,87,2826.32,...,87,384.94,131.75,87,43.17,46.21,87,0.19,0.24,0.51 (0.39)
1,-1.00,"(500.0, 1000.0]",80,18.75,12.77,80,56.71,67.96,80,2811.00,...,80,944.19,127.70,80,45.79,54.33,80,0.34,0.27,0.49 (0.39)
2,-1.00,"(1000.0, 2000.0]",80,23.75,15.86,80,65.03,68.40,80,2811.00,...,80,1711.38,292.60,80,52.44,54.70,80,0.42,0.23,0.51 (0.36)
3,-1.00,"(2000.0, 5000.0]",130,31.54,17.54,130,73.80,71.83,130,2811.00,...,130,3651.92,1096.81,130,59.42,57.45,130,0.52,0.22,0.53 (0.34)
4,-1.00,"(5000.0, inf]",73,41.78,13.47,73,61.60,44.89,73,2792.74,...,73,12037.67,5920.87,73,49.67,35.92,73,0.63,0.20,0.6 (0.32)
5,0.05,"(0.0, 500.0]",87,16.67,10.96,87,53.39,57.79,87,2826.32,...,87,384.94,131.75,87,43.17,46.21,87,0.29,0.27,0.49 (0.38)
6,0.05,"(500.0, 1000.0]",80,18.75,12.77,80,56.71,67.96,80,2811.00,...,80,944.19,127.70,80,45.79,54.33,80,0.46,0.24,0.48 (0.38)
7,0.05,"(1000.0, 2000.0]",80,23.75,15.86,80,65.03,68.40,80,2811.00,...,80,1711.38,292.60,80,52.44,54.70,80,0.52,0.23,0.49 (0.33)
8,0.05,"(2000.0, 5000.0]",130,31.54,17.54,130,73.80,71.83,130,2811.00,...,130,3651.92,1096.81,130,59.42,57.45,130,0.57,0.23,0.53 (0.34)
9,0.05,"(5000.0, inf]",73,41.78,13.47,73,61.60,44.89,73,2792.74,...,73,12037.67,5920.87,73,49.67,35.92,73,0.64,0.20,0.6 (0.32)


In [168]:
pd.set_option('display.max_rows', df.shape[0]+1)
v="r_ex_2b"
# display(df[df['V']=="r_ex_5b"].sort_values(by=['seed','N_nodes','alpha'], axis=0))
# print((df[df['V']==v].groupby(by=['seed']).size()))
# print(df[df['V']==v].groupby(by=['N_nodes']).size())
# print(df[df['V']==v].groupby(by=['alpha']).size())
v="r_ex_5b"

# print((df[df['V']==v].groupby(by=['seed']).size()))
# print(df[df['V']==v].groupby(by=['N_nodes']).size())
# print(df[df['V']==v].groupby(by=['alpha']).size())

# v="r_ex_1b_50"

print((df[df['V']==v].groupby(by=['seed']).size()))
print(df[df['V']==v].groupby(by=['N_nodes']).size())
print(df[df['V']==v].groupby(by=['alpha']).size())

seed
0       30
10      30
100     30
1000    30
2000    30
3000    30
4000    30
5000    30
6000    30
7000    30
dtype: int64
N_nodes
10    100
20    100
50    100
dtype: int64
alpha
50     60
100    60
200    60
300    60
500    60
dtype: int64


### Recostruction Experiment 50% known

In [169]:
import re
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import load_pickle, random_stability

verbose = False
verbose2 = False
nodes = '([\w]+)'
seeds = '([\w]+)'
dsets = '([\w.]+)'
outfs = str(1)
infs = str(1)
thetas = '([-+]?([0-9]+)(\.[0-9]+)?)'

# ## Versions
# # 'v3' reconstrunction toy experiment - 84%
# # 'p1' = partial toy experiment - 96%
# # 'r2' = reconstruction toy experiment, 20n 4b - 0%
# # 'r3' = reconstruction toy experiment, 10n 4b - 0%
# # 'r4' = reconstruction toy experiment, 10n 1b - 0%
# # 'r5' = reconstruction toy experiment, 15n 2b - ?
# # 'r6' = reconstruction toy experiment, 10n 1b - 32hidden
# # 'r7' = reconstruction toy experiment, 20n 4b - 0%
# # 'r_ex3' = reconstruction experiment, 10n 20n 50n 1b 20%known
# # 'r_ex_2b' = reconstruction experiment, 10n 20n 50n 2b 20%known
# # 'r_ex_5b' = reconstruction experiment, 10n 20n 50n 5b 20%known
# # 'r_ex_1b_50' = reconstruction experiment, 10n 20n 50n 2b 50%known
version = '(r_ex_1b_50)'

known_p = 0.5

folder = './results/'
# CASTLE/results/Nested1FoldCASTLE.Reg.Synth.10.625.r_ex2.pkl

count = 0
res = []

for filename in os.listdir(folder):
    match = re.search(f'Nested1FoldCASTLE.Reg.Synth.{nodes}.{dsets}.{version}.pkl$', filename)
    if match != None:
        count += 1
        
        report = load_pickle(os.path.join(folder, filename), verbose=False)
        for key, value in report.items():
            # print(key, value)
            t = [[match.group(3)],[value[i] for i in ['theta', 'n_nodes', 'N_edges', 'seed', 'data_size', 'MSE', 'MAE','right','matching']]]
            res.append([i for sublist in t for i in sublist])
            # print(res)
print("Count=",count)

df = pd.DataFrame(res)
df.columns = ['V','Type', 'N_nodes', 'N_edges', 'seed', 'Size', 'MSE', 'MAE','right','matching']
df['alpha'] = ((df['Size'].astype(int)*0.8)/df['N_nodes'].astype(int)).astype(int)
df['branch'] = np.round(((df['N_edges'].astype(int))/df['N_nodes'].astype(int)).astype(float),0).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 5)])
df['branch_bin'] = pd.cut(df['branch'], bins)

df['alpha2'] = ((df['Size'].astype(int)*0.8)/df['branch'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 1000), (1000, 2000), (2000, 5000), (5000,np.Inf)])
df['alpha3'] = pd.cut(df['alpha2'], bins)

df['alpha4'] = ((df['Size'].astype(int)*0.8)/df['N_edges'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 50), (50, 100), (100, 150), (150, 200), (200,np.Inf)])
df['alpha5'] = pd.cut(df['alpha4'], bins)

df['rebase_den'] = (df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))
df['rebased'] = (df['matching'].astype(int)/(df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))).clip(upper= 1 )

df = df[df['alpha'] != 1000]
# display(df)

    
plot_ly_by('alpha', df, x_desc="s = N/|V| (Dataset Size / Number of Nodes)")

plot_ly_by('N_nodes', df, x_desc="|V| = Number of Nodes in G")

plot_ly_by('N_nodes', df[df['alpha'] == 100], x_desc="|V| = Number of Nodes in G")

plot_ly_by('branch', df, x_desc="b = |E|/|V| (Number of Edges / Number of Nodes)")
plot_ly_by('branch_bin', df, x_desc="b = |E|/|V| (Number of Edges / Number of Nodes)")

plot_ly_by('alpha3', df, x_desc="Dataset Size / b (b=N Edges/N Nodes)")

plot_ly_by('alpha5', df, x_desc="N/|E| (Dataset Size / N Edges)")


# plot_ly_by('Size', df[df['N_nodes'] == 10], x_desc="Dataset Size")

Count= 15


,Type,alpha,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,50,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,26.67,1.60,30,25.63,17.35,30,0.18,0.23,0.65 (0.4)
1,-1.00,100,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,53.67,3.12,30,25.63,17.35,30,0.46,0.35,0.55 (0.34)
2,-1.00,200,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,108.07,6.34,30,25.63,17.35,30,0.83,0.20,0.53 (0.31)
3,-1.00,300,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,161.90,9.70,30,25.63,17.35,30,0.83,0.20,0.53 (0.31)
4,-1.00,500,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,270.37,16.18,30,25.63,17.35,30,0.82,0.22,0.53 (0.32)
5,0.05,50,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,26.67,1.60,30,25.63,17.35,30,0.40,0.35,0.63 (0.39)
6,0.05,100,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,53.67,3.12,30,25.63,17.35,30,0.81,0.21,0.54 (0.33)
7,0.05,200,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,108.07,6.34,30,25.63,17.35,30,0.90,0.17,0.54 (0.33)
8,0.05,300,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,161.90,9.70,30,25.63,17.35,30,0.88,0.21,0.54 (0.32)
9,0.05,500,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,270.37,16.18,30,25.63,17.35,30,0.93,0.14,0.54 (0.33)


,Type,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,Sizemean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,10,50,17.5,0.93,50,2811,2494.75,50,2875,...,50,131.30,93.07,50,9.1,0.54,50,0.59,0.40,0.59 (0.37)
1,-1.00,20,50,37.1,1.05,50,2811,2494.75,50,5750,...,50,123.68,87.29,50,18.7,0.46,50,0.52,0.38,0.46 (0.32)
2,-1.00,50,50,97.7,1.11,50,2811,2494.75,50,14375,...,50,117.42,82.71,50,49.1,0.71,50,0.76,0.23,0.61 (0.3)
3,0.05,10,50,17.5,0.93,50,2811,2494.75,50,2875,...,50,131.30,93.07,50,9.1,0.54,50,0.78,0.31,0.6 (0.37)
4,0.05,20,50,37.1,1.05,50,2811,2494.75,50,5750,...,50,123.68,87.29,50,18.7,0.46,50,0.70,0.36,0.47 (0.34)
5,0.05,50,50,97.7,1.11,50,2811,2494.75,50,14375,...,50,117.42,82.71,50,49.1,0.71,50,0.87,0.16,0.61 (0.28)


,Type,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,Sizemean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,10,10,17.5,0.97,10,2811,2603.26,10,1250,...,10,56.6,3.10,10,9.1,0.57,10,0.36,0.31,0.6 (0.39)
1,-1.00,20,10,37.1,1.10,10,2811,2603.26,10,2500,...,10,53.5,1.78,10,18.7,0.48,10,0.23,0.20,0.46 (0.33)
2,-1.00,50,10,97.7,1.16,10,2811,2603.26,10,6250,...,10,50.9,0.74,10,49.1,0.74,10,0.78,0.24,0.6 (0.29)
3,0.05,10,10,17.5,0.97,10,2811,2603.26,10,1250,...,10,56.6,3.10,10,9.1,0.57,10,0.78,0.27,0.6 (0.39)
4,0.05,20,10,37.1,1.10,10,2811,2603.26,10,2500,...,10,53.5,1.78,10,18.7,0.48,10,0.79,0.21,0.42 (0.29)
5,0.05,50,10,97.7,1.16,10,2811,2603.26,10,6250,...,10,50.9,0.74,10,49.1,0.74,10,0.88,0.14,0.6 (0.29)


,Type,branch,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,2,150,26.67,17.05,150,50.77,34.27,150,2811,...,150,124.13,87.39,150,25.63,17.12,150,0.62,0.36,0.56 (0.34)
1,0.05,2,150,26.67,17.05,150,50.77,34.27,150,2811,...,150,124.13,87.39,150,25.63,17.12,150,0.78,0.30,0.56 (0.34)


,Type,branch_bin,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,"(0, 1]",0,NaN,NaN,0,NaN,NaN,0,NaN,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,nan (nan)
1,-1.00,"(1, 2]",150,26.67,17.05,150,50.77,34.27,150,2811.0,...,150,124.13,87.39,150,25.63,17.12,150,0.62,0.36,0.56 (0.34)
2,-1.00,"(2, 5]",0,NaN,NaN,0,NaN,NaN,0,NaN,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,nan (nan)
3,0.05,"(0, 1]",0,NaN,NaN,0,NaN,NaN,0,NaN,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,nan (nan)
4,0.05,"(1, 2]",150,26.67,17.05,150,50.77,34.27,150,2811.0,...,150,124.13,87.39,150,25.63,17.12,150,0.78,0.30,0.56 (0.34)
5,0.05,"(2, 5]",0,NaN,NaN,0,NaN,NaN,0,NaN,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,nan (nan)


,Type,alpha3,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,"(0.0, 500.0]",30,13.33,4.79,30,24.03,9.45,30,2811,...,30,37.13,14.16,30,12.30,4.63,30,0.15,0.24,0.62 (0.41)
1,-1.00,"(500.0, 1000.0]",20,15.00,5.13,20,27.30,10.11,20,2811,...,20,83.85,31.45,20,13.90,4.95,20,0.54,0.39,0.51 (0.34)
2,-1.00,"(1000.0, 2000.0]",30,26.67,17.29,30,50.77,34.74,30,2811,...,30,101.40,61.09,30,25.63,17.35,30,0.68,0.29,0.56 (0.35)
3,-1.00,"(2000.0, 5000.0]",50,30.00,16.90,50,57.42,34.02,50,2811,...,50,174.00,93.08,50,28.94,17.01,50,0.81,0.21,0.52 (0.31)
4,-1.00,"(5000.0, inf]",20,50.00,0.00,20,97.70,1.13,20,2811,...,20,204.35,52.54,20,49.10,0.72,20,0.86,0.12,0.59 (0.28)
5,0.05,"(0.0, 500.0]",30,13.33,4.79,30,24.03,9.45,30,2811,...,30,37.13,14.16,30,12.30,4.63,30,0.41,0.36,0.62 (0.41)
6,0.05,"(500.0, 1000.0]",20,15.00,5.13,20,27.30,10.11,20,2811,...,20,83.85,31.45,20,13.90,4.95,20,0.86,0.22,0.49 (0.33)
7,0.05,"(1000.0, 2000.0]",30,26.67,17.29,30,50.77,34.74,30,2811,...,30,101.40,61.09,30,25.63,17.35,30,0.82,0.24,0.55 (0.34)
8,0.05,"(2000.0, 5000.0]",50,30.00,16.90,50,57.42,34.02,50,2811,...,50,174.00,93.08,50,28.94,17.01,50,0.90,0.14,0.53 (0.32)
9,0.05,"(5000.0, inf]",20,50.00,0.00,20,97.70,1.13,20,2811,...,20,204.35,52.54,20,49.10,0.72,20,0.91,0.13,0.6 (0.29)


,Type,alpha5,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,"(0.0, 50.0]",33,28.79,17.81,33,55.15,35.94,33,2797.88,...,33,28.79,6.98,33,27.85,17.98,33,0.24,0.29,0.63 (0.4)
1,-1.00,"(50.0, 100.0]",27,24.07,16.23,27,45.41,32.37,27,2827.04,...,27,54.07,3.02,27,22.93,16.11,27,0.42,0.34,0.57 (0.34)
2,-1.00,"(100.0, 150.0]",30,26.67,17.29,30,50.77,34.74,30,2811.00,...,30,108.07,6.34,30,25.63,17.35,30,0.83,0.20,0.53 (0.31)
3,-1.00,"(150.0, 200.0]",30,26.67,17.29,30,50.77,34.74,30,2811.00,...,30,161.90,9.70,30,25.63,17.35,30,0.83,0.20,0.53 (0.31)
4,-1.00,"(200.0, inf]",30,26.67,17.29,30,50.77,34.74,30,2811.00,...,30,270.37,16.18,30,25.63,17.35,30,0.82,0.22,0.53 (0.32)
5,0.05,"(0.0, 50.0]",33,28.79,17.81,33,55.15,35.94,33,2797.88,...,33,28.79,6.98,33,27.85,17.98,33,0.44,0.36,0.6 (0.38)
6,0.05,"(50.0, 100.0]",27,24.07,16.23,27,45.41,32.37,27,2827.04,...,27,54.07,3.02,27,22.93,16.11,27,0.81,0.22,0.56 (0.33)
7,0.05,"(100.0, 150.0]",30,26.67,17.29,30,50.77,34.74,30,2811.00,...,30,108.07,6.34,30,25.63,17.35,30,0.90,0.17,0.54 (0.33)
8,0.05,"(150.0, 200.0]",30,26.67,17.29,30,50.77,34.74,30,2811.00,...,30,161.90,9.70,30,25.63,17.35,30,0.88,0.21,0.54 (0.32)
9,0.05,"(200.0, inf]",30,26.67,17.29,30,50.77,34.74,30,2811.00,...,30,270.37,16.18,30,25.63,17.35,30,0.93,0.14,0.54 (0.33)


In [170]:
import re
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import load_pickle, random_stability

verbose = False
verbose2 = False
nodes = '([\w]+)'
seeds = '([\w]+)'
dsets = '([\w.]+)'
outfs = str(1)
infs = str(1)
thetas = '([-+]?([0-9]+)(\.[0-9]+)?)'

# ## Versions
# # 'v3' reconstrunction toy experiment - 84%
# # 'p1' = partial toy experiment - 96%
# # 'r2' = reconstruction toy experiment, 20n 4b - 0%
# # 'r3' = reconstruction toy experiment, 10n 4b - 0%
# # 'r4' = reconstruction toy experiment, 10n 1b - 0%
# # 'r5' = reconstruction toy experiment, 15n 2b - ?
# # 'r6' = reconstruction toy experiment, 10n 1b - 32hidden
# # 'r7' = reconstruction toy experiment, 20n 4b - 0%
# # 'r_ex3' = reconstruction experiment, 10n 20n 50n 1b
# # 'r_ex_2b' = reconstruction experiment, 10n 20n 50n 2b

version = '(r_ex_1b_50)'

folder = './results/'
# CASTLE/results/Nested1FoldCASTLE.Reg.Synth.10.625.r_ex2.pkl

count = 0
res = []

for filename in os.listdir(folder):
    match = re.search(f'Nested1FoldCASTLE.Reg.Synth.{nodes}.{dsets}.{version}.pkl$', filename)
    if match != None:
        count += 1
        
        report = load_pickle(os.path.join(folder, filename), verbose=False)
        for key, value in report.items():
            # print(key, value)
            t = [[match.group(3)],[value[i] for i in ['theta', 'n_nodes', 'N_edges', 'seed', 'data_size', 'MSE', 'MAE','right','matching']]]
            res.append([i for sublist in t for i in sublist])
            # print(res)
print("Count=",count)

df = pd.DataFrame(res)
df.columns = ['V','Type', 'N_nodes', 'N_edges', 'seed', 'Size', 'MSE', 'MAE','right','matching']
df['alpha'] = ((df['Size'].astype(int)*0.8)/df['N_nodes'].astype(int)).astype(int)
df['branch'] = np.round(((df['N_edges'].astype(int))/df['N_nodes'].astype(int)).astype(float),0).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 5)])
df['branch_bin'] = pd.cut(df['branch'], bins)

df['alpha2'] = ((df['Size'].astype(int)*0.8)/df['branch'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 1000), (1000, 2000), (2000, 5000), (5000,np.Inf)])
df['alpha3'] = pd.cut(df['alpha2'], bins)
df['rebase_den'] = (df['N_edges'].astype(int)-(df['N_edges']*0.2).astype(int))
df['rebased'] = (df['matching'].astype(int)/(df['N_edges'].astype(int)-(df['N_edges']*0.2).astype(int))).clip(upper= 1 )

df = df[df['alpha'] != 1000]

v="r_ex_1b_50"

print((df[df['V']==v].groupby(by=['seed']).size()))
print(df[df['V']==v].groupby(by=['N_nodes']).size())
print(df[df['V']==v].groupby(by=['alpha']).size())

Count= 15
seed
0       30
10      30
100     30
1000    30
2000    30
3000    30
4000    30
5000    30
6000    30
7000    30
dtype: int64
N_nodes
10    100
20    100
50    100
dtype: int64
alpha
50     60
100    60
200    60
300    60
500    60
dtype: int64


# Adult Tau Optimisation Chart

In [171]:
import re
dset='100000'
theta = '(([+-])?\d(\.\d+)?)'
version = '(adult2|adult)'
folder = "./results/adjmats/"
list_edges = []
for filename in os.listdir(folder):
#     print(filename)
    if version in ['v3','p1']:
        match = re.search(f'W_est.{dset}.([\w]+).([\w.]+).{version}.pkl$', filename)
    else:
        match = re.search(f'W_est.{dset}.(\d).(\d).(.*?).{version}.pkl$', filename)
    if match:
        # print( match.groups())
#         print(filename)
        out_f, in_f, theta, version = match.groups()
        edges = sum(load_pickle(os.path.join(folder,filename), verbose=False).flatten()>0)
        list_edges.append((float(theta), out_f, in_f, edges))
#         print(file)
# pd.DataFrame(list_edges)
n_edges = pd.DataFrame(list_edges).groupby(0, as_index=False).agg({3 : ['mean', 'std']})
# n_edges[n_edges[0]==0.006]
n_edges.columns = n_edges.columns.droplevel()
n_edges.columns = ['theta','N_edges','Std']
n_edges
# n_edges.loc[-1] = [-1,405.12,10.401602]  # adding a row
# n_edges.index = n_edges.index + 1  # shifting index
# n_edges = n_edges.sort_index()  # sorting by index
# n_edges
## uncostrained -1.000,405.120000,10.401602

ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
folder = "./results/"
name = 'adult2'
filename = f"Nested5FoldCASTLE.Reg.Synth.100000.{name}.pkl"
describe = load_pickle(os.path.join(folder,filename))
# [c[9] for c in describe]
# describe
# pd.DataFrame([(describe[c]['theta'],c[9],describe[c]['fold'],describe[c]['accuracy']) for c in describe]).sort_values(by=[0,1,2]).groupby([0,1], as_index=False).agg({2 : ['count','max'], 3 : ['mean', 'std']})
agg_stats = pd.DataFrame([(describe[c]['theta'],c[9],describe[c]['fold'],describe[c]['accuracy']) for c in describe]).groupby(0, as_index=False).agg({2 : ['count','max'], 3 : ['mean', 'std']})
agg_stats.columns = ['theta', 'count','folds',  'accuracy_mean', 'accuracy_std']

display(agg_stats)

# n_edges = pd.DataFrame([(theta, nx.from_numpy_matrix(DAG_retreive_np(W_est, theta), create_using=nx.DiGraph, parallel_edges=False).number_of_edges()) for theta in agg_stats['theta']])

# n_edges.loc[n_edges['theta']==-1,'theta'] = sub
# n_edges.loc[n_edges[0]==sub,1] = nx.from_numpy_matrix(W_est, create_using=nx.DiGraph, parallel_edges=False).number_of_edges()

# display(n_edges)
# is_DAG = pd.DataFrame([(theta, nx.is_directed_acyclic_graph(nx.from_numpy_matrix(DAG_retreive_np(W_est, theta), create_using=nx.DiGraph, parallel_edges=False))) for theta in agg_stats['theta']])
# is_DAG.columns = ['theta', 'is_DAG']
# n_edges_DAG = n_edges.merge(is_DAG, on='theta')

# n_edges_DAG#.columns = ['theta', 'N_edges', 'is_DAG']

# agg_stats=agg_stats.merge(n_edges_DAG,on='theta')
agg_stats=agg_stats.merge(n_edges,on='theta')
agg_stats= agg_stats.drop_duplicates(subset=['N_edges'])

sub = -0.005
agg_stats.loc[agg_stats['theta']==-1,'theta'] = sub

# filter_theta = 0.04
# agg_stats = agg_stats[agg_stats['theta']<=filter_theta]
# agg_stats

# agg_stats.loc[-1] = [-0.001,5,5,0.723377,0.009611,405.12,10.401602]#,True]  # adding a row
# agg_stats.index = agg_stats.index + 1  # shifting index
# agg_stats = agg_stats.sort_index()  # sorting by index
agg_stats

Loaded PICKLE from ./results/Nested5FoldCASTLE.Reg.Synth.100000.adult2.pkl


,theta,count,folds,accuracy_mean,accuracy_std
0,-1.000,10,5,0.759720,0.015492
1,0.000,10,5,0.782926,0.013687
2,3.185,10,5,0.764664,0.000216
3,6.370,5,5,0.764868,0.000000
4,9.555,5,5,0.764868,0.000000
5,12.740,5,5,0.764868,0.000000
6,19.110,5,5,0.764868,0.000000
7,25.480,5,5,0.764868,0.000000
8,47.775,5,5,0.764868,0.000000


,theta,count,folds,accuracy_mean,accuracy_std,N_edges,Std
0,-0.005,10,5,0.759720,0.015492,210,0.000000
1,0.000,10,5,0.782926,0.013687,105,0.000000
2,3.185,10,5,0.764664,0.000216,18,5.270463
3,6.370,5,5,0.764868,0.000000,12,0.000000
4,9.555,5,5,0.764868,0.000000,11,0.000000
5,12.740,5,5,0.764868,0.000000,5,0.000000
6,19.110,5,5,0.764868,0.000000,3,0.000000
7,25.480,5,5,0.764868,0.000000,2,0.000000
8,47.775,5,5,0.764868,0.000000,1,0.000000


In [ ]:

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=agg_stats['theta'], y=agg_stats['accuracy_mean'], name="Accuracy",         
    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=agg_stats['accuracy_std'],
            visible=True)
            ),
    secondary_y=False, 
)

fig.add_trace(
    go.Scatter(x=agg_stats['theta'], y=agg_stats['N_edges'], name="Edges",
#     error_y=dict(
#             type='data', # value of error bar given in data coordinates
#             array=agg_stats['Std'],
#             visible=True)
              ),
    secondary_y=True,
)

# fig.add_trace(
#     go.Scatter(x=agg_stats['theta'], y=agg_stats['DAG_mean'], name="DAG score",         
#     error_y=dict(
#             type='data', # value of error bar given in data coordinates
#             array=agg_stats['DAG_std'],
#             visible=True)
#             ),
#     secondary_y=False,
# )

# Add figure title
fig.update_layout(
    title='',
#     {
#         'text': "Shift in Accuracy on Test Set and Number of DAG Edges by tau",
#         'y':0.85,
#         'x':0.5,
#         'xanchor': 'center',
#         'yanchor': 'top'},
    legend={
        'y':0.87,
        'x':0.7,
        'orientation':"h",
        'xanchor': 'center',
        'yanchor': 'bottom'},
    template='plotly_white',
    autosize=True,
#     width=100,
    height=400,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=10,
        pad=0
    ),
    font=dict(
        # family="Courier New, monospace",
        size=18,
        # color="Black"
    )    
)

# Set x-axis title
fig.update_xaxes(showgrid=True,title={'text':r'tau','font':{'size':18}})
# font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
# Set y-axes titles
fig.update_yaxes(title={'text':"Accuracy",'font':{'size':18}}, nticks=13,secondary_y=False)
fig.update_yaxes(showgrid=False,nticks=10,zeroline=False, title={'text':"Number of Edges",'font':{'size':18}}, secondary_y=True)

# fig.update_layout(
#     yaxis = dict(
#         tickmode = 'linear',
#         tick0 = 0.51,
#         dtick = 0.03
#     )
# )
fig.show()

In [ ]:
verbose = False
verbose2 = False
nodes = '([\w]+)'
seeds = '([\w]+)'
dsets = '([\w.]+)'
outfs = str(1)
infs = str(1)
thetas = '([-+]?([0-9]+)(\.[0-9]+)?)'

# ## Versions
# # 'v3' reconstrunction toy experiment - 84%
# # 'p1' = partial toy experiment - 96%
# # 'r2' = reconstruction toy experiment, 20n 4b - 0%
# # 'r3' = reconstruction toy experiment, 10n 4b - 0%
# # 'r4' = reconstruction toy experiment, 10n 1b - 0%
# # 'r5' = reconstruction toy experiment, 15n 2b - ?
# # 'r6' = reconstruction toy experiment, 10n 1b - 32hidden
# # 'r7' = reconstruction toy experiment, 20n 4b - 0%
# # 'r_ex3' = reconstruction experiment, 10n 20n 50n 1b
# # 'r_ex_2b' = reconstruction experiment, 10n 20n 50n 2b

version = '(r_ex3|r_ex_2b|r_ex_5b)'
# version = '(r_ex_5b|a)'

folder = './results/'
# CASTLE/results/Nested1FoldCASTLE.Reg.Synth.10.625.r_ex2.pkl

count = 0
res = []

for filename in os.listdir(folder):
    match = re.search(f'Nested1FoldCASTLE.Reg.Synth.{nodes}.{dsets}.{version}.pkl$', filename)
    if match != None:
        count += 1
        
        report = load_pickle(os.path.join(folder, filename), verbose=False)
        for key, value in report.items():
            # print(key, value)
            t = [[match.group(3)],[value[i] for i in ['theta', 'n_nodes', 'N_edges', 'seed', 'data_size']],[value[i].total_seconds()/60 for i in ['timestamp']]]
            res.append([i for sublist in t for i in sublist])
            # print(res)
print("Count=",count)
df = pd.DataFrame(res)
df.columns = ['V','Type', 'N_nodes', 'N_edges', 'seed', 'Size', 'timestamp']
# display(df)
df = df[df['seed']!=7000]


x = 'Size'
ds1 = df.groupby(['Type',x], as_index=False).agg([ 'count','mean','std']).round(2).reset_index()
ds1.columns = list(map(''.join, ds1.columns.values))
# mses['text'] = mses[['MSEmean','MSEstd']].apply(lambda x : '{} ({})'.format(x[0],x[1]), axis=1)#mses['MSEmean'].astype(str) + mses['MSEstd'].astype(str) 
# mses[x] = mses[x].astype(str)
# display(ds1)


import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": False}]])

ds1C = ds1[ds1['Type']==-1]
# Add traces
fig.add_trace(
    go.Scatter(x=ds1C[x], y=ds1C['timestampmean'], name=f"CASTLE+",         
    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=ds1C['timestampstd'],
            visible=True)
            ),
    secondary_y=False, 
)

ds1I = ds1[ds1['Type']==0.05]
# Add traces
fig.add_trace(
    go.Scatter(x=ds1I[x], y=ds1I['timestampmean'], name=f"Injected",         
    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=ds1I['timestampstd'],
            visible=True)
            ),
    secondary_y=False, 
)
# Add figure title
fig.update_layout(
    margin=dict(
        l=20,
        r=20,
        b=20,
        t=20,
        pad=1
    ),
    title={
        'text': f"Run Time per Dataset Size",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    legend={
        'y':-0.18,
        'x':0.8,
        'orientation':"h",
        'xanchor': 'center',
        'yanchor': 'bottom'},
    template='plotly_white',
    font=dict(
        # family="Courier New, monospace",
        size=18,
        # color="Black"
    )    

)

# Set x-axis title
fig.update_xaxes(title_text="Dataset Size")

# Set y-axes titles
fig.update_yaxes(title_text="Minutes per run", secondary_y=False)
# fig.update_yaxes(title_text="Number of Edges in DAG", secondary_y=True)


fig.show()

Count= 48
